## loading data/libs

In [148]:
import pandas as pd
import calendar
from bokeh.charts import output_notebook, Scatter, Bar, show, output_file, Line, BoxPlot, Scatter
from bokeh.plotting import figure
from bokeh.io import hplot
output_notebook() 

Loading BokehJS ...

In [149]:
INPUT="data/device_failure.csv" 
dataset = pd.read_csv(INPUT,index_col=[0,1],parse_dates=[0])

In [150]:
label_dset = dataset[["failure"]]


### checking devices

In [151]:
total_failures_per_device = label_dset.groupby(level=1).agg(sum)
total_failures_per_device["failure"].value_counts()

0    1062
1     106
Name: failure, dtype: int64

Each device fail at least once

~10% device failing

'only' 106 positive points

### checking Dates

In [152]:
dates = label_dset.index.get_level_values(0)
print "Range: from %s to %s" % (dates.min(), dates.max())


total_failures_per_date = label_dset.groupby(level=0).agg(sum)
print
print " n failures per date"
print str(total_failures_per_date["failure"].value_counts())
print
print "total: %i failures for %i days" % (total_failures_per_date["failure"].sum(), 
                                          total_failures_per_date[total_failures_per_date.failure >0].count())

Range: from 2015-01-01 00:00:00 to 2015-11-02 00:00:00

 n failures per date
0    228
1     54
2     19
3      2
8      1
Name: failure, dtype: int64

total: 106 failures for 76 days


In [170]:
d = test.resample("M")
d.agg({"device":np.unique})
#d.unique()

,device
date,
2015-01-31,25032
2015-02-28,19500
2015-03-31,19833
2015-04-30,12012
2015-05-31,11330
2015-06-30,10469
2015-07-31,10531
2015-08-31,8346
2015-09-30,4470


In [174]:
from bokeh.plotting import figure
data =total_failures_per_date.resample("M").sum() 
test = label_dset.reset_index("device").resample("M").agg(lambda d : d.nunique())
data["n_devices"] = test["device"]
data["failure_ratio_pm"] = data["failure"] / data["n_devices"] * 1000
data.index = (calendar.month_abbr[i] for i in data.index.month)
l = Line(
    data["failure_ratio_pm"],
    title="failures per Month",
    ylabel="n_failures", 
    xlabel="month"
)
show(l)

In [176]:
l = Line(
    data["n_devices"],
    title="n devices seen per Month",
    ylabel="n_failures", 
    xlabel="month"
)
show(l)

In [7]:
weekday_dset = total_failures_per_date.copy()
weekday_dset.index = ["%i:%s" % (i,calendar.day_name[i]) for i in total_failures_per_date.index.weekday]

per_day = weekday_dset.groupby(level=0).sum()

print "failures per weekday"

per_day.sort_index()


failures per weekday


,failure
0:Monday,27
1:Tuesday,18
2:Wednesday,15
3:Thursday,22
4:Friday,12
5:Saturday,8
6:Sunday,4


- Long term trend with more failures in the past
- Less  failures over the weekend
- The absence of weekend could be explained by maintenance hapening only during workweek (hence explaing more failures on monday


### Per Device description 

In [19]:
import numpy as np
dates = label_dset.swaplevel().reset_index("date")
dd= dates["date"]
devices = pd.DataFrame({"min_date":dd.groupby(level=0).min(),"failure":dates["failure"].groupby(level=0).sum()})
devices["max_date"] =  dd.groupby(level=0).max()
devices["n_lines"] = dd.groupby(level=0).count()
devices["n_days"] = (devices["max_date"] - devices["min_date"] ) /np.timedelta64(1, 'D') +1
devices["missing_values"] = devices["n_days"] - devices["n_lines"]

In [9]:
devices["min_date"].value_counts()

2015-01-01    1163
2015-05-06       4
2015-01-27       1
Name: min_date, dtype: int64

### checking the nb devices per month. this is better done above

In [10]:
#pd.DataFrame({"n_devices":devices["max_date"].dt.month.value_counts().sort_index()})
montlhy_devices = pd.DataFrame({"n_devices":devices["max_date"].dt.month.value_counts().sort_index()})
montlhy_devices.index = [calendar.month_abbr[i] for i in montlhy_devices.index]
montlhy_devices

,n_devices
Jan,399
Feb,46
Mar,184
Apr,112
May,72
Jun,6
Jul,15
Aug,150
Sep,38
Oct,115


### bucketing the n devices with missing day data

In [11]:
i = ( (devices["missing_values"] //20)*20).value_counts()
#i = ( (devices["missing_values"])).value_counts(bins=10)
i.index.name = "n missing days"
pd.DataFrame({"n devices":i.sort_index()})

,n devices
n missing days,
-0.0,1077
20.0,26
40.0,21
60.0,8
80.0,3
100.0,28
120.0,4
140.0,1


In [12]:
i = devices["n_days"].value_counts(bins=10).sort_index()
i.index.name='n_days'
b = Bar(pd.DataFrame(
    {"n_devices":i}),
    xlabel="n days",
   title="devices distributed by ndays"
       )
show(b)

In [94]:
data = withdate
d = data[data["failure"]>0]["date"]
d[0]

Timestamp('2015-01-19 00:00:00')

In [118]:
failing_devices = devices[devices["failure"]>0].index
failing_devices_t = pd.DataFrame({"failure":label_dset["failure"].unstack().filter(items=failing_devices).unstack()}).dropna()
def max_date(date):
    return np.max(date)

def failing_date(date):
    data = withdate.ix[date.index]
    return data[data["failure"]>0]["date"][0]

withdate = failing_devices_t.reset_index(level=1)
max_vs_failingdates = withdate.groupby(level=0).agg( {"date": [ max_date, failing_date ],"failure": np.sum})
max_vs_failingdates.columns = max_vs_failingdates.columns.droplevel()
max_vs_failingdates["td"] = (max_vs_failingdates["max_date"] - max_vs_failingdates["failing_date"]) / np.timedelta64(1, 'D')
print
print "dt in days between first failure and end of measurement :"
print max_vs_failingdates["td"].value_counts()
print
print "n failures"
print max_vs_failingdates["sum"].value_counts()



dt in days between first failure and end of measurement :
0.0     101
2.0       2
30.0      1
1.0       1
12.0      1
Name: td, dtype: int64

n failures
1.0    106
Name: sum, dtype: int64


In [146]:
print "looking at weird failures"
weird_devices = max_vs_failingdates[max_vs_failingdates["td"] > 0]
weirdos = failing_devices_t.reset_index(level=1).ix[set(weird_devices.index)]
print weirdos.set_index("date",append=True).unstack(level="device").to_string()

looking at weird failures
            failure                                    
device     S1F0GPFZ S1F136J0 W1F0KCP2 W1F0M35B W1F11ZG9
date                                                   
2015-01-01      0.0      0.0      0.0      0.0      0.0
2015-01-02      0.0      0.0      0.0      0.0      0.0
2015-01-03      0.0      0.0      0.0      0.0      0.0
2015-01-04      0.0      0.0      0.0      0.0      0.0
2015-01-05      0.0      0.0      0.0      0.0      0.0
2015-01-06      0.0      0.0      0.0      0.0      0.0
2015-01-07      0.0      0.0      0.0      0.0      0.0
2015-01-08      0.0      0.0      0.0      0.0      0.0
2015-01-09      0.0      0.0      0.0      0.0      0.0
2015-01-10      0.0      0.0      0.0      0.0      0.0
2015-01-11      0.0      0.0      0.0      0.0      0.0
2015-01-12      0.0      0.0      0.0      0.0      0.0
2015-01-13      0.0      0.0      0.0      0.0      0.0
2015-01-14      0.0      0.0      0.0      0.0      0.0
2015-01-15      0.0   

 - indetified a list of devices, which are still measured after having failed.
 
three hypothesis:
 - The device is still functionnal after maintenance
 - The failure was a fluke
 - The measurement thereafter are false
 
 ==> if we cannot distinguish between these hypothesis, need to remove these devices from the dataset 